In [32]:
import glob
import os
from pathlib import Path
from datetime import datetime
import rasterio
import logging
import numpy as np
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)

def filename_to_unixtime(filename):
    
    components = Path(filename).stem.split('_')
    date_str = components[2]
    time_str = components[3]

    # Convert to datetime object
    dt = datetime.strptime(date_str + time_str, "%Y%m%d%H%M%S")
    
    # Convert datetime object to Unix timestamp
    return int(dt.timestamp())

filenames = list(Path(os.getcwd() + '/rasters').glob('*.tif'))

# Sort filenames based on their Unix time
sorted_filenames = sorted(filenames, key=filename_to_unixtime)

#Size of temporal dimension
n_files = len(sorted_filenames)

In [30]:
# Read the first file to get the shape of each 2D layer
src = rasterio.open(sorted_filenames[0])
mtr, rho_temp = src.read([1, 2])
vx_temp, vy_temp = src.read([3, 4])


#src.transform returns a 3x3 matrix that defines the georeferencing of the raster data. units: m
#provides the affine transformation that maps pixel locations in (row, col) coordinates to (x, y) spatial positions
dx = src.transform[0]
dy = -src.transform[4]  
dt = 10 / 60 

logging.info("dx: {}, dy: {}, dt: {}".format(dx, dy, dt))

INFO:root:dx: 8032.112543557559, dy: 4379.351044515573, dt: 0.16666666666666666


In [35]:
# Get the shape of one layer and initialize 3D array
height, width = rho_temp.shape

# Initialize empty 3D arrays
rho = np.empty((n_files, height, width))
vx = np.empty((n_files, height, width))
vy = np.empty((n_files, height, width))

# Loop through all the files and fill the arrays
for i in tqdm(range(n_files)):  # starting from 0 now
    with rasterio.open(sorted_filenames[i]) as src:
        mtr, rho_temp = src.read([1, 2])
        vx_temp, vy_temp = src.read([3, 4])

        rho[i, :, :] = rho_temp
        vx[i, :, :] = vx_temp
        vy[i, :, :] = vy_temp

# Convert vx and vy from m/s to km/h
vx *= 1000 / 60 / 60
vy *= 1000 / 60 / 60

logging.info("rho array shape: {}".format(rho.shape))

100%|██████████████████████████████████████████████████████████████████████████████████| 349/349 [00:13<00:00, 25.88it/s]
INFO:root:rho array shape: (349, 800, 800)


In [62]:
def pad_with_nan(matrix, dimension):
    """Pad an n-dimensional matrix with NaN values along the specified dimension."""
    if dimension < 0 or dimension >= len(matrix.shape):
        raise ValueError(f"Dimension must be between 0 and {len(matrix.shape) - 1}.")

    pad_width = [(0, 0) for _ in matrix.shape]
    pad_width[dimension] = (1, 1)  # Adjusted for padding on both sides

    return np.pad(matrix, pad_width, mode='constant', constant_values=np.nan)

def pad_with_zeros(matrix, padding_dims=(1, 2)):
    """
    Pad a 3D matrix with 0 values along the specified dimensions.
    - matrix (ndarray): The input 3D matrix.
    - padding_dims (tuple of int, optional): The dimensions to pad. Default is (1, 2).

    Returns:
    - ndarray: The padded matrix.
    """
    if not all(0 <= dim < 3 for dim in padding_dims):
        raise ValueError(f"Dimensions in padding_dims must be among 0, 1, and 2.")

    pad_width = [(0, 0) for _ in matrix.shape]
    for dim in padding_dims:
        pad_width[dim] = (1, 1)

    return np.pad(matrix, pad_width, mode='constant', constant_values=0)

def simple_moving_avg_3D(matrix, axis=0):
    """A simplified moving average for a 3D matrix."""
    if axis == 0:  # z-axis
        pairs = np.stack([matrix[:-1, :, :], matrix[1:, :, :]])
    elif axis == 1:  # y-axis
        pairs = np.stack([matrix[:, :-1, :], matrix[:, 1:, :]])
    elif axis == 2:  # x-axis
        pairs = np.stack([matrix[:, :, :-1], matrix[:, :, 1:]])
    else:
        raise ValueError("Invalid axis.")
    
    return np.nanmean(pairs, axis=0)


def calculate_phi(rho, vx, vy, dx, dy):
    # Compute the flux Φ = vρ at +/- 1/2 grid cell.
    # First, add a nan layer in y or x direction
    Phiy_pad = pad_with_nan(rho * vy * dx / 1000,1)
    Phix_pad = pad_with_nan(rho * vx * dy / 1000,2)

    # Compute the flux at +/- 1/2 even if either previous or next cell is NaN.
    Phiy_h = simple_moving_avg_3D(Phiy_pad, axis=1)
    Phix_h = simple_moving_avg_3D(Phix_pad, axis=2)
    
    return Phix_h, Phiy_h


In [63]:
# Usage
Phix_h, Phiy_h = calculate_phi(rho, vx, vy, dx, dy)

In [64]:
logging.info("Phix_h: {}  Phiy_h: {}".format(Phix_h.shape, Phiy_h.shape))

INFO:root:Phix_h: (349, 800, 801)  Phiy_h: (349, 801, 800)


In [65]:
Phiy_h_0 = pad_with_zeros(Phiy_h)
Phix_h_0 = pad_with_zeros(Phix_h)


In [58]:
logging.info("Phix_h_0: {}  Phiy_h_0: {}".format(Phix_h_0.shape, Phiy_h_0.shape))

INFO:root:Phix_h_0: (349, 802, 802)  Phiy_h_0: (349, 802, 802)


In [66]:
Phiy_h_0[np.isnan(Phiy_h_0)] = 0
Phix_h_0[np.isnan(Phix_h_0)] = 0

# Compute the differences
dPhiy = np.diff(Phiy_h_0, axis=1)
dPhix = np.diff(Phix_h_0, axis=2)

In [67]:
logging.info("dPhiy: {}  dPhix: {}".format(dPhiy.shape, dPhix.shape))

INFO:root:dPhiy: (349, 802, 802)  dPhix: (349, 802, 802)


In [68]:
F = (dPhiy + dPhix) * dt

In [74]:
F[0][0]

array([ 0.        , -4.26525902, -4.29316374, -4.32178963, -4.35112462,
       -4.38115487, -4.41186201, -4.44322724, -4.47522787, -4.50783883,
       -4.54103186, -4.57477445, -4.60903274, -4.64376868, -4.67894071,
       -4.7145038 , -4.75041054, -4.78660914, -4.82304684, -4.85966563,
       -4.89640714, -4.93320936, -4.97000951, -5.00674245, -5.04334269,
       -5.0797444 , -5.11588211, -5.15169105, -5.18710772, -5.22207119,
       -5.2565222 , -5.29040356, -5.3236638 , -5.35625297, -5.3881256 ,
       -5.41923997, -5.44955826, -5.4790466 , -5.50767353, -5.53541326,
       -5.56224175, -5.58813764, -5.61308173, -5.63705882, -5.66005486,
       -5.68205703, -5.70305447, -5.72303803, -5.74199932, -5.75993109,
       -5.77692414, -5.79353717, -5.80983528, -5.82579273, -5.84138178,
       -5.85654713, -5.87038116, -5.88244733, -5.89272694, -5.90120238,
       -5.90785796, -5.91268096, -5.91566005, -5.91678681, -5.91605396,
       -5.91345813, -5.90899682, -5.90267272, -5.89448721, -5.88

In [82]:
os.getcwd()

'/Users/at744/clo/bitbucket/birdcast/aws_batch_monitoring'

In [83]:
#Matlab comparison

import scipy.io
data = scipy.io.loadmat('F.mat')
matrices = data['F']

# Transpose the matrix such that time is the first dimension
mat = matrices.transpose(2, 0, 1)

In [84]:
mat[0][0]

array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.     